# Smoothing BreakfastClub 

In [ ]:
from nilearn import image 
import glob, os

# get data
sessions = ["ses-1", "ses-2"]
data_path = "/projects/niblab/bids_projects/Experiments/BreakfastClub/derivatives"
SUBS = sorted(glob.glob(os.path.join(data_path, "sub-*")))
for sub_path in SUBS:
    for ses in sessions:
        ## path exists == session exists, continue with conversion
        ## get functional images 
        FUNCS = glob.glob(os.path.join(sub_path, "func", "*_brain.nii.gz"))
        for epi_file in FUNCS:
            filename = epi_file.split("/")[-1]
            subID = filename.split("_")[0]
            task = filename.split("_")[2]
            ses = filename.split("_")[1]
            new_filename = os.path.join(sub_path, "func/%s_%s_%s_smoothed.nii.gz"%(subID, ses, task))
            print(new_filename)
            mean_func = image.mean_img(epi_file)
            smoothed_img = image.smooth_img(mean_func, fwhm=5)
            smoothed_img.to_filename(new_filename)
                


In [ ]:
from multiprocessing import Pool
import glob
import pandas as pd
import argparse
import os
import subprocess
import datetime
import shutil
     

In [ ]:
def skull_strip(sub, func_path):
    print(">>>>---> starting bet on ", sub )
    try:
        for nifti in glob.glob(os.path.join(func_path, '%s/func/*ses-2*desc-preproc_bold.nii.gz'%sub)):
            # make our variables
            filename = nifti.split("/")[-1].split(".")[0]
            bet_name=filename+'_brain'
            bet_output = os.path.join(func_path,"%s/func"%sub, bet_name)
            print("SKULL STRIP, NEW FILE TO BE MADE: ", bet_name)
            if os.path.exists(bet_output + '.nii'):
                print(bet_output + ' exists, skipping \n')
            else:
                print("Running bet on ", nifti)
                bet_cmd=("bet %s %s -F -m -f %s"%(nifti, bet_output, "0.6"))
                print(">>>-----> BET COMMAND:", bet_cmd)
                os.system(bet_cmd)
            
    except:
        pass

    
deriv_path= "/projects/niblab/bids_projects/Experiments/BreakfastClub/derivatives"
subjects = sorted(glob.glob(os.path.join(deriv_path, "sub-*")))

    
for path in subjects:
    sub_id = path.split("/")[-1]
    skull_strip(sub_id,deriv_path)






        
        
        
        

In [ ]:
def fd_check(sub, outfile, out_bad_bold_list, derivatives_dir):
    func_path = os.path.join(derivatives_dir, sub, "func")
    print(">>>>---> Starting motion correction on ", sub)
    motion_assessment_path=os.path.join(derivatives_dir, sub, 'func','motion_assessment')
    if not os.path.exists(motion_assessment_path):
        os.makedirs(motion_assessment_path)
    try:
# iterate over nifti file
        for nifti in glob.glob(os.path.join(func_path, '*_brain.nii*')):
            filename=nifti.split('.')[0]
            file = filename.split("/")[-1]
            outlier_path = "%s/%s_outlier_output.txt"%(motion_assessment_path, file)
            plot_path = "%s/%s_fd_plot"%(motion_assessment_path, file)
            confound_path = "%s/%s_confound.txt"%(motion_assessment_path, file)
            #need to get identifier for tasks and runs --rn for bevel, need to specify for versatility 
            # set comparison param
            nvols_cmd="fslnvols " + nifti
            volume = subprocess.check_output(nvols_cmd, shell=True, encoding="utf-8")
            volume = volume.strip()
            comparator = int(volume) *.25
            ## RUN 'fsl_motion_outliers' TO RETRIEVE MOTION CORRECTION ANALYSIS
            outlier_cmd = "fsl_motion_outliers -i %s  -o %s --fd --thresh=%s -p %s -v > %s"%(filename, confound_path, "0.9", plot_path, outlier_path)
            print(">>-->  RUNNING FSL MOTION OUTLIERS ")
            print("COMMAND NVOLS: ", nvols_cmd)
            print("OUTLIER CMD: ", outlier_cmd)
            os.system(outlier_cmd)
        ## EXAMINE OUTLIER FILE AND GRAB RELEVANT DATA 
            with open(outlier_path, 'r') as f:
                lines=f.readlines()
                statsA = lines[1].strip("\n") #maskmean
                statsB = lines[3].strip("\n") #metric range
                statsC = lines[4].strip("\n") #outliers found
                if int(statsC.split(" ")[1])  > 0:
                    statsD = lines[6].strip("\n") #spikes found
                else:
                    statsD = "\n"
            f.close()
        ## GRAB MOTION CORRECTION PLOT AND WRITE PLOT & INFO TO HTML
            plotz=plot_path+".png"
            FILEINFO="""<p><font size=6> <b>{CURR_FILENAME} </b></font><br>"""
            CURR_FILEINFO = FILEINFO.format(CURR_FILENAME=file)
            outfile.write(CURR_FILEINFO)
            INFO="""<p><font size=6>{A} <br><b>{B}<b><br>{C}<br><b>{D}</b><br><br>"""
            CURR_INFO= INFO.format(A=statsA, B=statsB, C=statsC, D=statsD)
            outfile.write(CURR_INFO)
            PLOT="""<IMG SRC=\"{PLOTPATH}\" WIDTH=100%><br><br>"""
            CURR_PLOT = PLOT.format(PLOTPATH=plotz)
            outfile.write(CURR_PLOT)
            print(">>>>----> ADDING PLOT TO HTML")
                ## ADD FILE FOR GOOD SUBJECT 
        # --sometimes you have a great subject who didn't move
            if os.path.isfile(confound_path)==False:
                os.system("touch %s"%confound_path)
                
        ## CHECK FOR BAD SUBJECTS: ABOVE OUR THRESHOLD
        # how many columns are there = how many 'bad' points
            check = subprocess.check_output("grep -o 1 %s | wc -l"%(confound_path), shell=True)
            num_scrub = [int(s) for s in check.split() if s.isdigit()]
            print("NUM SCRUB: ", str(num_scrub[0]), "\n")
            if num_scrub[0] > comparator: #if the number in check is greater than num_scrub then we don't want it
                with open(out_bad_bold_list, "a") as myfile: #making a file that lists all the bad ones
                    myfile.write("%s/%s\n"%(derivatives_dir, file))
                    print("wrote bad file")
                myfile.close()
    except FileNotFoundError:   
        print("FILE IS EMPTY, PASSING")

datestamp=datetime.datetime.now().strftime("%Y-%m-%d-%H_%M_%S")

outhtml = os.path.join(deriv_path,'bold_motion_QA_%s.html'%(datestamp))
out_bad_bold_list = os.path.join(deriv_path,'%s_TEST.txt'%(datestamp))
        
    # OPEN HTML FILE IF CASE TRUE
outfile = open(outhtml, 'a')
TITLE="""<p><font size=7> <b> Motion Correction Check</b></font><br>"""
outfile.write("%s"%TITLE)


    
for path in subjects:
    sub_id = path.split("/")[-1]
    fd_check(sub_id, outfile,
             out_bad_bold_list, deriv_path)

In [ ]:
def write_files(filename, moco_df, outputdir):
    # iterate through the motion correction data frame by columns, writing individual columns to individual files
    for col in moco_df.columns:
        file= "%s_%s.txt"%(filename, col)
        output_path=os.path.join(outputdir, file)
        print("Writing to file, ", output_path)
        moco_df[col].to_csv(output_path, header=False, index=False)


def get_motion_parameters(sub):
    errors = []
    motion_assessment_path =os.path.join( deriv_path, sub, "func/motion_assessment" ) 
    moco_avail = len(glob.glob(os.path.join(motion_assessment_path, "motion_parameters/*.txt")))
    #print(sub,moco_avail)
    if moco_avail < 0:
        pass
    else:
        try:
        #print("--------------> GETTING MOCOS FOR SUBJECT: ", sub)
            outputdir = os.path.join(motion_assessment_path, "motion_parameters")
            if not os.path.exists(os.path.join(outputdir, 'motion_parameters')):
                os.makedirs(os.path.join(outputdir, 'motion_parameters'))
        #os.chdir(filepath)
            confound_path_s1 = "/projects/niblab/bids_projects/Experiments/BreakfastClub/fmriprep_lin/fmriprep/%s/ses-1/func/%s_ses-1_task-resting_desc-confounds_regressors.tsv"%(sub, sub)
            confound_path_s2 = "/projects/niblab/bids_projects/Experiments/BreakfastClub/fmriprep_lin/fmriprep/%s/ses-2/func/%s_ses-2_task-resting_desc-confounds_regressors.tsv"%(sub, sub)
        #print("-------> GRABBING FILES:", confound_path_s1)
            df_s1 = pd.read_table(confound_path_s2)
            #print(df_s1.columns.values)
    # print(df_s1[['motion_outlier01', 'motion_outlier02', 'motion_outlier03', 'motion_outlier04', 'motion_outlier05', 'motion_outlier06']])
#        moco_s1=df_s1[['X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ']]
            moco_s1= df_s1[['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z']]
            moco_s1.columns = ["moco0", "moco1", "moco2", "moco3", "moco4", "moco5"]
            #print(moco_s1)
            print("DATAFRAME: \n ", moco_s1.head())
            filename = confound_path_s2.split("/")[-1].split("_desc-confounds_regressors")[0]
            print("FILENAME: ", filename)
            write_files(filename, moco_s1, outputdir)
        except:
            print("error subject ", sub)
            pass
        """
        #print("RUN: ", run)
    except FileNotFoundError as not_found:
        print("********************FILE NOT FOUND: ", not_found.filename)
        if sub not in errors:
            errors.append(sub)
        #print("ERRORS ", errors)
        #print("ERRORS SORTED ", sorted(errors))
    errors = sorted(errors)
    for err in errors:
        #print("ERROR" + err)
        file = basedir+"/error_files_moco.txt"
        with open(file, 'a') as f:
            f.write("----------> FILE NOT FOUND FOR SUBJECT: " + err  + "\n")
            f.close()"""
        
        
deriv_path= "/projects/niblab/bids_projects/Experiments/BreakfastClub/derivatives"
subjects = sorted(glob.glob(os.path.join(deriv_path, "sub-*")))

    
for path in subjects:
    sub_id = path.split("/")[-1]
    get_motion_parameters(sub_id)
    

    